## Leiab liigutamisverbidega laused korpusest, rakendab neile samu filtreid mis andmestiku koostamisel ja kirjutab laused eraldi faili

In [1]:
import pyconll #conllu failide lugemise teek

In [4]:
#funktsioon, mis tagastab kõik laused, kus otsitav verb esineb vormides, mis pole välja filtreeritud
def verb_korpusest(verb, korpus):
    margendus = []
    for lause in korpus:
        for sona in lause:
            if sona.lemma == verb and sona.upos == 'V' and lause not in margendus:
                #filtreerime välja verbid, mis on järgmistes vormides:
                #umbisikuline
                if (('Voice' in sona.feats.keys() and 'imps' in sona.feats['Voice']) == False 
                #käskiv kõneviis
                and ('Mood' in sona.feats.keys() and 'imper' in sona.feats['Mood']) == False
                #supiin
                and ('VerbForm' in sona.feats.keys() and 'sup' in sona.feats['VerbForm']) == False
                #da-infinitiiv
                and ('VerbForm' in sona.feats.keys() and 'inf' in sona.feats['VerbForm']) == False
                #v-kesksõna
                and ('VerbForm' in sona.feats.keys() and 'partic' in sona.feats['VerbForm'] 
                and 'Tense' in sona.feats.keys() and 'pres' in sona.feats['Tense']) == False
                #gerundiiv e des-vorm    
                and ('VerbForm' in sona.feats.keys() and 'ger' in sona.feats['VerbForm']) == False
                and (sona.deprel != 'aux')):
                    #kui verb pole ühesgi välja filtreeritavas vormis, lisame selle lause listi
                    margendus.append(lause) 
    return margendus

In [5]:
#filtreerime välja ühendverbid ja verbid, millel on alluv aux
def filter_alluvad(verb, laused):
    head_laused = []
    for lause in laused:
        for sona in lause:
            #kui sõna ülemus on meie verb ja sõna pole abiverb ega ühendverbi osis
            if ('H_lemma' in sona.deps.keys() and verb in sona.deps['H_lemma'] 
                and sona.deprel != 'aux' and sona.deprel != 'compound:prt'): 
                if lause not in head_laused:
                    head_laused.append(lause)
    return head_laused

In [14]:
#kirjutame filtrid läbinud laused vastavalt verbile eraldi failidesse
verbid = ['riputama', 'tirima', 'loopima', 'paiskama', 'raputama', 'torkama', 'sättima', 'toppima', 'laduma', 'poetama', 
          'pistma', 'asetama', 'langetama', 'sõidutama', 'paigutama', 'vedama', 'heitma', 'lükkama', 'tõukama', 'seadma',
          'keerama', 'viskama', 'tõmbama', 'tõstma', 'liigutama', 'viima', 'tooma', 'panema']

koik_laused = [] #siia salvestame kõik filtrid läbinud liigutamisverbide laused

for verb in verbid:
    sisu = ''
    #loeme kogu korpuse sisse
    sisu = pyconll.iter_from_file('data/korpusefailid/tasak.conll')
    #otsime korpusest lauseid oma verbiga ja filtreerime vormi järgi halvad välja
    margendus = verb_korpusest(verb, sisu)
    #filtreerime välja ühendverbiga ja abiverbidega laused
    filtreeritud = filter_alluvad(verb, margendus)
    koik_laused += filtreeritud
    failitee = 'data/paringutulemused/' + verb + '.conllu'
    #kirjutame filtrid läbinud laused faili
    with open(failitee, 'w', encoding='utf-8') as f:
        for line in filtreeritud:
            f.write(f"{line.conll()}\n\n")

In [17]:
#kirjutame kõik liigutamisverbide laused eraldi faili
with open('data/paringutulemused/koik_verbid.conll', 'w', encoding='utf-8') as f:
    for line in koik_laused:
        f.write(f"{line.conll()}\n\n")